**2TDSP(R/S/T) - PRIMEIRO SEMESTRE**

AULA 01: APRENDIZADO NÃO SUPERVISIONADO - AGRUPAMENTO (CLUSTERING)

EXEMPLO DE UTILIZAÇÃO DO ALGORITMO K-MEANS

**Situação Problema:**

Necessidade da criação de um modelo preditivo para agrupamento de clientes por similaridade, com o objetivo de compreender o comportamento dos clientes e sua relação com o consumo de energia.

Base de dados disponível em:
https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Carga de dados
df = pd.read_csv('/content/household_power_consumption.txt', delimiter = ';')
df.head()

Dados disponíveis:

1. **Date**: Data no formato dd/mm/aaaa

2. **Time**: hora no formato hh:mm:ss

3. **Global_active_power**: potência ativa média global por minuto da casa (em kilowatt)

4. **Global_reactive_power**: potência reativa média global por minuto doméstica (em kilowatt)

5. **Voltage**: tensão média por minuto (em volts)

6. **Global_intensity**: intensidade atual média global por minuto (em ampères)

7. **Sub_metering_1**: submedição de energia nº 1 (em watt-hora de energia ativa) correspondente à cozinha, contendo uma máquina de lavar louça, um forno e um micro-ondas.

8. **Sub_metering_2**: submedição de energia nº 2 (em watt-hora de energia ativa) correspondente à lavanderia, contendo máquina de lavar, secadora, geladeira e luminária.

9. **Sub_metering_3**: submedição de energia nº 3 (em watt-hora de energia ativa), correspondente a um aquecedor elétrico e um aparelho de ar condicionado.

Análise Exploratória

In [ ]:
# Linhas e colunas do Data Frame
df.shape

In [ ]:
# Tipo de dados
df.dtypes

In [ ]:
# Informações gerais do dataset
df.info()

In [ ]:
# Verificação de dados faltantes
df.isnull().values.any()

In [ ]:
# Verificação de dados nulos
df.isnull().sum()

Pré-processamento

In [ ]:
# Remoção de registros com valores vazios e das colunas Date e Time que não serão necessárias.
df = df.iloc[:, 2:9].dropna()
df.head()

In [ ]:
# Verificação de dados faltantes
df.isnull().values.any()

In [ ]:
# Verificação de dados nulos
df.isnull().sum()

In [ ]:
# Criação de um array com os valores dos atributos (Esses serão nossos dados de entrada)
df_atrib = df.values
df_atrib

In [ ]:
# Geração de amostra de 1% dos dados.
df, amostra = train_test_split(df_atrib, train_size = 0.01)

In [ ]:
df.shape

Redução de dimensionalidade:

Utilizaremos o PCA (Principal Component Analysis ou Análise da Componente Principal) para reduzir a dimensionalidade dos dados.

A ideia do PCA é diminuir a dimensão do conjunto de dados original escolhendo um número menor de componentes para projeção sobre estas.
O processo de redução de dimensão facilita processamento e armazenamento de dados.

Transformaremos 7 variáveis em 2 variáveis, com base na matriz de covariância.

Documentação:
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
pca = PCA(n_components = 2).fit_transform(df)

Utilização do Algoritmo K-Means

In [ ]:
# Determinando um range do hiperparâmetro "k" do K-Means
k_range = range(1,12)
k_range

In [ ]:
# Aplicando o K-Means para cada valor de K:
k_means_var = [KMeans(n_clusters = k).fit(pca) for k in k_range]

In [ ]:
# Escolhendo um valor de K inicial igual a 8 para avaliação do modelo
modelo_01 = KMeans(n_clusters = 8)
modelo_01.fit(pca)

In [ ]:
# Silhoutte Score
labels = modelo_01.labels_
silhouette_score(pca, labels, metric = 'euclidean')

In [ ]:
# Treinando o modelo com K = 9
modelo_02 = KMeans(n_clusters = 9)
modelo_02.fit(pca)

In [ ]:
# Silhoutte Score
labels = modelo_02.labels_
silhouette_score(pca, labels, metric = 'euclidean')

In [ ]:
# Treinando o modelo com K = 10
modelo_03 = KMeans(n_clusters = 10)
modelo_03.fit(pca)

In [ ]:
# Silhoutte Score
labels = modelo_03.labels_
silhouette_score(pca, labels, metric = 'euclidean')

Utilizando os Clusters do modelo escolhido.

In [ ]:
nomes = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

In [ ]:
# Mapeamento de clusters
clusters = pd.DataFrame(df, columns = nomes)

# Escolhendo uma das colunas e transformando em numérico:
clusters['Global_active_power'] = pd.to_numeric(clusters['Global_active_power'])
clusters['cluster'] = modelo_01.labels_
clusters

In [ ]:
# Exemplo: Calculando a média de consumo de energia por cluster
clusters.groupby('cluster')['Global_active_power'].mean()